In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np

In [2]:
url="https://books.toscrape.com/catalogue/"
webpage = requests.get(url)
webpage.text

'<html>\r\n<head><title>403 Forbidden</title></head>\r\n<body>\r\n<center><h1>403 Forbidden</h1></center>\r\n<hr><center>nginx/1.21.6</center>\r\n</body>\r\n</html>\r\n'

In [3]:
def scrape_books(url):
    books = []
    while url:
        response = requests.get(url)
        if response.status_code != 200:
            print("Failed to retrieve page")
            break
        soup = BeautifulSoup(response.text, 'html.parser')
        book_list = soup.find_all('article', class_='product_pod')
        for book in book_list:
            title = book.h3.a.attrs['title']
            price = book.find('p', class_='price_color').text[1:]
            availability = book.find('p', class_='instock availability').text.strip()
            rating = book.p.attrs['class'][1]
            books.append([title, price, availability, rating])
        next_page = soup.find('li', class_='next')
        if next_page:
            url = "https://books.toscrape.com/catalogue/" + next_page.a.attrs['href']
        else:
            url = None
    return books

In [4]:
start_url = "https://books.toscrape.com/catalogue/page-1.html"
book_data = scrape_books(start_url)

In [5]:
df = pd.DataFrame(book_data, columns=['Title', 'Price (GBP)', 'Availability', 'Rating'])

In [6]:
df.to_csv("books_dataset.csv", index=False)

print("Scraping complete. Dataset saved as 'books_dataset.csv'")

Scraping complete. Dataset saved as 'books_dataset.csv'
